1. Go through this Post: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ (Links to an external site.)

2. Add these improvements to the final code described in the post:
  1. Predict 500 characters only
  2. Remove all the punctuation from the source text
  3. Train the model on padded sequences (Links to an external site.) rather than random sequences of characters. **pending**
  4.Train the model for 100 epochs
  5. Add dropout to the input layer, remove it from the layer before dense layer. Use Dropout value of 0.1 everywhere
  6. Submit!



In [0]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
#Loading google drive to retrive data and store training weights
from google.colab import drive
drive.mount('/content/gdrive')

#preparing path for LSTM_exploration dir in drive
import os
corpus = os.path.join("/content/gdrive/My Drive", "LSTM_exploration")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# load ascii text and covert to lowercase
filename = os.path.join(corpus, "wonderland.txt") #"wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

**Original LSTM implimentation from mentioned artical's last step**

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163781
Total Vocab:  59


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  163681


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

In [0]:
# normalize
X = X / float(n_vocab)

In [0]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0727 12:47:36.510854 140028818990976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 12:47:36.549080 140028818990976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 12:47:36.556473 140028818990976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 12:47:36.861359 140028818990976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 12:47:36.870846 

In [0]:
# define the checkpoint
filepath=os.path.join(corpus, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5")
#"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
# fit the model
model.fit(X, y, epochs=50, batch_size=256, callbacks=callbacks_list)

Epoch 1/50
163681/163681 [==============================] - 169s 1ms/step - loss: 2.7930

Epoch 00001: loss improved from inf to 2.79305, saving model to /content/gdrive/My Drive/LSTM_exploration/weights-improvement-01-2.7930-bigger.hdf5
Epoch 2/50
163681/163681 [==============================] - 169s 1ms/step - loss: 2.6358

Epoch 00002: loss improved from 2.79305 to 2.63584, saving model to /content/gdrive/My Drive/LSTM_exploration/weights-improvement-02-2.6358-bigger.hdf5
Epoch 3/50
163681/163681 [==============================] - 169s 1ms/step - loss: 2.5083

Epoch 00003: loss improved from 2.63584 to 2.50827, saving model to /content/gdrive/My Drive/LSTM_exploration/weights-improvement-03-2.5083-bigger.hdf5
Epoch 4/50
163681/163681 [==============================] - 168s 1ms/step - loss: 2.3952

Epoch 00004: loss improved from 2.50827 to 2.39525, saving model to /content/gdrive/My Drive/LSTM_exploration/weights-improvement-04-2.3952-bigger.hdf5
Epoch 5/50
163681/163681 [==========

In [0]:
# load the network weights

filename = os.path.join(corpus, "weights-improvement-50-1.3639-bigger.hdf5")
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [0]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" uite silent for a
minute or two, which gave the pigeon the opportunity of adding, 'you're
looking fo "
w a gond deal tiis sime it to tea the datter of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole partering out of the tort of the table, and the white rabbit ceeen wery slle time the court. 
and she whole parter

**Changing model for suggested changes in step 2**

In [0]:
#removing punctuation from text
#s.translate(str.maketrans('', '', string.punctuation))
import string
raw_text1 = raw_text.translate(str.maketrans('', '', string.punctuation))

# create mapping of unique chars to integers, and a reverse mapping
chars1 = sorted(list(set(raw_text1)))
char_to_int1 = dict((c, i) for i, c in enumerate(chars1))
int_to_char1 = dict((i, c) for i, c in enumerate(chars1))

# summarize the loaded data
n_chars1 = len(raw_text1)
n_vocab1 = len(chars1)
print ("Total Characters: ", n_chars1)
print ("Total Vocab: ", n_vocab1)

Total Characters:  154861
Total Vocab:  39


In [0]:
# prepare the dataset of input to output pairs encoded as integers; removed punctuations
seq_length = 100
dataX1 = []
dataY1 = []

for i in range(0, n_chars1 - seq_length, 1):
	seq_in1 = raw_text1[i:i + seq_length]
	seq_out1 = raw_text1[i + seq_length]
	dataX1.append([char_to_int1[char1] for char1 in seq_in1])
	dataY1.append(char_to_int1[seq_out1])


n_patterns1 = len(dataX1)
print ("Total Patterns: ", n_patterns1)

Total Patterns:  154761


In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
# define the document
#text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of the vocabulary
words = set(text_to_word_sequence(raw_text1))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
result = one_hot(raw_text1, round(vocab_size*1.3))
print(result)

3251
[4131, 790, 3856, 3877, 2732, 4030, 2715, 34, 1807, 550, 813, 3206, 1308, 3261, 1530, 4090, 2942, 2979, 424, 1951, 891, 2863, 553, 1338, 1951, 3653, 3698, 383, 1863, 3271, 3785, 306, 3785, 1468, 102, 4030, 3785, 3146, 3261, 2976, 4090, 3261, 2379, 543, 3812, 3099, 553, 550, 813, 102, 2957, 424, 2498, 1911, 3856, 3877, 2732, 4030, 1929, 34, 1807, 3278, 2192, 1442, 1304, 2212, 813, 1526, 2126, 2192, 2321, 3716, 1185, 960, 1346, 854, 3853, 1538, 683, 829, 4090, 550, 2379, 543, 813, 3856, 3877, 2732, 4030, 3856, 3877, 2732, 4030, 34, 1807, 3261, 2708, 3844, 2561, 3526, 370, 1099, 271, 3261, 4199, 4190, 2467, 2321, 829, 3839, 3560, 3852, 4090, 3363, 2715, 803, 2408, 3796, 3261, 4054, 2863, 4090, 1887, 4034, 829, 2261, 3432, 102, 1322, 1751, 3660, 3322, 2941, 3261, 4081, 803, 2408, 2467, 3823, 2874, 3785, 3660, 1951, 1407, 102, 1576, 2732, 3785, 2863, 674, 3206, 3261, 1530, 4090, 1933, 4081, 1973, 4190, 1506, 1407, 102, 1576, 2760, 1751, 2467, 2503, 2732, 803, 3399, 2271, 3374, 131, 337

In [0]:
# reshape X to be [samples, time steps, features]
X1 = numpy.reshape(dataX1, (n_patterns1, seq_length, 1))
# normalize
X1 = X1 / float(n_vocab1)
# one hot encode the output variable
y1 = np_utils.to_categorical(dataY1)

In [0]:
# define other LSTM model with dropout at input layer and reducing dropout to 0.1
model1 = Sequential()
model1.add(Dropout(0.1))
model1.add(LSTM(256, input_shape=(X1.shape[1], X1.shape[2]), return_sequences=True))
model1.add(Dropout(0.1))
model1.add(LSTM(256))
model1.add(Dense(y1.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
#model1.summary()

W0731 01:50:36.916958 140067935790976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 01:50:36.953404 140067935790976 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# define the checkpoint
filepath=os.path.join(corpus, "new-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5")
#"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
# fit the model1 for 100 epoch
model1.fit(X1, y1, epochs=100, batch_size=256, callbacks=callbacks_list)

W0730 14:23:07.864768 139881456039808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 14:23:07.868364 139881456039808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0730 14:23:07.886922 139881456039808 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0730 14:23:07.905755 139881456039808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backe

Epoch 1/100
154761/154761 [==============================] - 169s 1ms/step - loss: 2.8382

Epoch 00001: loss improved from inf to 2.83820, saving model to /content/gdrive/My Drive/LSTM_exploration/new-weights-improvement-01-2.8382-bigger.hdf5
Epoch 2/100
154761/154761 [==============================] - 163s 1ms/step - loss: 2.6752

Epoch 00002: loss improved from 2.83820 to 2.67524, saving model to /content/gdrive/My Drive/LSTM_exploration/new-weights-improvement-02-2.6752-bigger.hdf5
Epoch 3/100
154761/154761 [==============================] - 161s 1ms/step - loss: 2.5500

Epoch 00003: loss improved from 2.67524 to 2.55003, saving model to /content/gdrive/My Drive/LSTM_exploration/new-weights-improvement-03-2.5500-bigger.hdf5
Epoch 4/100
154761/154761 [==============================] - 160s 1ms/step - loss: 2.4287

Epoch 00004: loss improved from 2.55003 to 2.42870, saving model to /content/gdrive/My Drive/LSTM_exploration/new-weights-improvement-04-2.4287-bigger.hdf5
Epoch 5/100
1547

In [0]:
# load the network weights

filename = os.path.join(corpus, "new-weights-improvement-100-1.3851-bigger.hdf5")
model1.load_weights(filename)
model1.compile(loss='categorical_crossentropy', optimizer='adam')

ValueError: ignored

In [0]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")